In [1]:
import pandas as pd
import openpyxl
import awswrangler as awr

pegando excel

In [2]:
excel_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"

df_base = pd.read_excel(excel_path, engine='openpyxl')



ajustes pontuais base

In [3]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "data_atualizacao",
    "situacao"
]

df_base = df_base[colunas_comuns]
df_base.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa','ponteiro', 'situacao', 'data_atualizacao'], inplace=True)
df_base.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa','ponteiro', 'data_atualizacao'], inplace=True)

df_base['data_vencimento'] = pd.to_datetime(df_base['data_vencimento'])
df_base['data_vencimento'] = df_base['data_vencimento'].dt.date 

df_base['data_baixa'] = pd.to_datetime(df_base['data_baixa'])
df_base['data_baixa'] = df_base['data_baixa'].dt.date 

df_base['data_emissao'] = pd.to_datetime(df_base['data_emissao'])
df_base['data_emissao'] = df_base['data_emissao'].dt.date 

tratando colunas nulas

In [4]:
df_base['data_baixa'] = df_base['data_baixa'].fillna(pd.Timestamp('1900-01-01'))
df_base['data_emissao'] = df_base['data_emissao'].fillna(pd.Timestamp('1900-01-01'))
df_base['data_vencimento'] = df_base['data_vencimento'].fillna(pd.Timestamp('1900-01-01'))
df_base['valor_titulo'] = df_base['valor_titulo'].fillna(0)
df_base['valor_baixa'] = df_base['valor_baixa'].fillna(0)
df_base['nosso_numero'] = df_base['nosso_numero'].fillna('NULL')
df_base['numero_boleto'] = df_base['numero_boleto'].fillna('NULL')

gerando base de inadimplência

In [5]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

df_inadimplentes.loc[:, 'data_atualizacao'] = df_inadimplentes['data_vencimento']
df_inadimplentes.loc[:, 'situacao']='INADIMPLENTE'


selecionando limite base

In [6]:
today = pd.to_datetime('today').date()
year = today.year
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento']> pd.to_datetime('2025-05-31')]
#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]

#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.year == year]




tratando colunas df_inadimplentes

In [7]:

for col in colunas_comuns:
    if col not in df_inadimplentes.columns:
        df_inadimplentes.loc[:, col] = pd.NA

df_inadimplentes = df_inadimplentes[colunas_comuns]

concatenando e removendo linhas inteiras duplicadas

In [8]:
df_composto_inadimplentes =  pd.concat([df_base, df_inadimplentes])
df_composto_inadimplentes = df_composto_inadimplentes.drop_duplicates(keep='first')


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_16900\3111717009.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_composto_inadimplentes =  pd.concat([df_base, df_inadimplentes])


criando listra que filtra apenas os ponteiros de inadimplentes

In [9]:
#df_composto_inadimplentes['data_vencimento'] = pd.to_datetime(df_composto_inadimplentes['data_vencimento'])
#df_composto_inadimplentes_mes = df_composto_inadimplentes[df_composto_inadimplentes['data_vencimento'].dt.month == month]
#df_composto_inadimplentes_mes = df_composto_inadimplentes[df_composto_inadimplentes['data_vencimento'].dt.year == year]

df_inadimplentes_lista = df_composto_inadimplentes.loc[df_composto_inadimplentes['data_vencimento'].notna() ,'ponteiro'].to_list()


gerando o relatório de pagamentos do mês e tratando colunas

In [10]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
df_pagamentos = df_pagamentos[colunas_comuns]

tratando colunas

In [11]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

In [12]:


df_pagamentos['data_baixa'] = pd.to_datetime(df_pagamentos['data_baixa'])
df_pagamentos['data_vencimento'] = pd.to_datetime(df_pagamentos['data_vencimento'])

df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
df_pagamentos = df_pagamentos[
    (df_pagamentos['data_baixa']> pd.to_datetime('2025-05-31'))&
    (df_pagamentos['data_vencimento']> pd.to_datetime('2025-05-31'))
    ]
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.month == month]
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.year == year]

df_pagamentos_inadimplencia = df_pagamentos[df_pagamentos['ponteiro'].isin(df_inadimplentes_lista)]


concatenando faturas baixadas dos vencidos deste mes com a base composta e atualizada

In [13]:
df_atualizado = pd.concat([df_composto_inadimplentes, df_pagamentos_inadimplencia])
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'situacao', 'data_atualizacao'], inplace=True) #eliminando faturas duplicadas
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'data_atualizacao'], inplace=True) #eliminando faturas que tenham a mesma data e identificação

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_16900\3147402727.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_atualizado = pd.concat([df_composto_inadimplentes, df_pagamentos_inadimplencia])


ajustando formatação

In [14]:

df_atualizado['data_vencimento'] = pd.to_datetime(df_atualizado['data_vencimento'])
df_atualizado['data_vencimento'] = df_atualizado['data_vencimento'].dt.date 

df_atualizado['data_baixa'] = pd.to_datetime(df_atualizado['data_baixa'])
df_atualizado['data_baixa'] = df_atualizado['data_baixa'].dt.date 

df_atualizado['data_emissao'] = pd.to_datetime(df_atualizado['data_emissao'])
df_atualizado['data_emissao'] = df_atualizado['data_emissao'].dt.date 

df_atualizado['data_atualizacao'] = pd.to_datetime(df_atualizado['data_atualizacao']).dt.date

df_atualizado = df_atualizado[colunas_comuns]

tratando colunas nulas

In [15]:
df_atualizado['data_baixa'] = df_atualizado['data_baixa'].fillna(pd.Timestamp('1900-01-01'))
df_atualizado['data_vencimento'] = df_atualizado['data_vencimento'].fillna(pd.Timestamp('1900-01-01'))
df_atualizado['data_emissao'] = df_atualizado['data_emissao'].fillna(pd.Timestamp('1900-01-01'))
df_atualizado['valor_titulo'] = df_atualizado['valor_titulo'].fillna(0)
df_atualizado['valor_baixa'] = df_atualizado['valor_baixa'].fillna(0)
df_atualizado['nosso_numero'] = df_atualizado['nosso_numero'].fillna('NULL')
df_atualizado['numero_boleto'] = df_atualizado['numero_boleto'].fillna('NULL')

retornando ao excel base atualizado

In [16]:
df_atualizado['data_emissao'] = pd.to_datetime(df_atualizado['data_emissao'], errors='coerce')
df_atualizado['data_emissao'] = df_atualizado['data_emissao'].dt.date

df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'situacao', 'data_atualizacao'], inplace=True) #eliminando faturas duplicadas
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'data_atualizacao'], inplace=True) #eliminando faturas que tenham a mesma data e identificação

excel_save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"
df_atualizado.to_excel(excel_save_path, engine='openpyxl', index=False, sheet_name='historico_faturas')

In [17]:
len(df_atualizado)
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'situacao', 'data_atualizacao'], inplace=True) #eliminando faturas duplicadas
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'data_atualizacao'], inplace=True) #eliminando faturas que tenham a mesma data e identificação
len(df_atualizado)

3571

teste

In [18]:
#path1 = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\teste.xlsx"
#df_atualizado.to_excel(path1, engine='openpyxl',index=False)